## IBKR POC

This is an experimental notebook where I follow the guide here:

https://www.pyquantnews.com/the-pyquant-newsletter/automate-trading-strategies-powerful-ib-api

With TWS open in the background, and the API enabled, this code enables historical and real-time stock data retrieval, as well as trade entry and exit

In [2]:
# %pip install dotenv
# %pip install praw
# %pip install ibapi

In [ ]:
import time
import threading
from typing import Dict, Optional
import pandas as pd

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import BarData

In [25]:
class TradingApp(EClient, EWrapper):

    def __init__(self) -> None:
        EClient.__init__(self, self)
        self.data: Dict[int, pd.DataFrame] = {}
        self.nextOrderId: Optional[int] = None

    def error(self, reqId: int, errorCode: int, errorString: str) -> None:
        print(f"Error: {reqId}, {errorCode}, {errorString}")

    def nextValidId(self, orderId: int) -> None:
        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def get_historical_data(self, reqId: int, contract: Contract) -> pd.DataFrame:
        self.data[reqId] = pd.DataFrame(columns=["time", "high", "low", "close"])
        self.data[reqId].set_index("time", inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime="",
            durationStr="1 D",
            barSizeSetting="1 min",
            whatToShow="MIDPOINT",
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(5)
        return self.data[reqId]

    def historicalData(self, reqId: int, bar: BarData) -> None:
        df = self.data[reqId]

        df.loc[
            pd.to_datetime(bar.date, unit="s"), 
            ["high", "low", "close"]
        ] = [bar.high, bar.low, bar.close]

        df = df.astype(float)

        self.data[reqId] = df

    @staticmethod
    def get_contract(symbol: str) -> Contract:
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        return contract

    def place_order(self, contract: Contract, action: str, order_type: str, quantity: int) -> None:
        order = Order()
        order.action = action
        order.orderType = order_type
        order.totalQuantity = quantity
        self.placeOrder(self.nextOrderId, contract, order)
        self.nextOrderId += 1
        print("Order placed")

In [ ]:
app = TradingApp()
app.connect("127.0.0.1", 7497, clientId=7)

threading.Thread(target=app.run, daemon=True).start()

In [27]:
while True:
    if isinstance(app.nextOrderId, int):
        print("connected")
        break
    else:
        print("waiting for connection")
        time.sleep(1)


nvda = TradingApp.get_contract("NVDA")

connected


In [28]:
data = app.get_historical_data(99, nvda)
data.tail()

,high,low,close
time,,,
2025-12-20 00:55:00,181.36,181.33,181.36
2025-12-20 00:56:00,181.36,181.27,181.31
2025-12-20 00:57:00,181.34,181.30,181.34
2025-12-20 00:58:00,181.37,181.34,181.37
2025-12-20 00:59:00,181.37,181.35,181.37


In [29]:
def donchian_channel(df: pd.DataFrame, period: int = 30) -> pd.DataFrame:
    df["upper"] = df["high"].rolling(window=period).max()
    df["lower"] = df["low"].rolling(window=period).min()
    df["mid"] = (df["upper"] + df["lower"]) / 2

    return df

In [30]:
period = 30

while True:

    print("Getting data for contract...")
    data = app.get_historical_data(99, nvda)

    if len(data) < period:
        print(f"There are only {len(data)} bars of data, skipping...")
        continue

    print("Computing the Donchian Channel...")
    donchian = donchian_channel(data, period=period)

    last_price = data.iloc[-1].close

    upper, lower = donchian[["upper", "lower"]].iloc[-1]

    print(f"Check if last price {last_price} is outside the channels {upper} and {lower}")

    if last_price >= upper:
        print("Breakout detected, going long...")
        app.place_order(nvda, "BUY", "MKT", 10)

    elif last_price <= lower:
        print("Breakout detected, going short...")
        app.place_order(nvda, "SELL", "MKT", 10)

Getting data for contract...
Computing the Donchian Channel...
Check if last price 181.37 is outside the channels 181.37 and 181.26
Breakout detected, going long...
Order placed
Getting data for contract...
Computing the Donchian Channel...
Check if last price 181.37 is outside the channels 181.37 and 181.26
Breakout detected, going long...
Order placed
Getting data for contract...


KeyboardInterrupt: 

In [31]:
app.disconnect()